## Package Imports

In [2]:
import pyaudio
import librosa
import logging
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from multiprocessing import Process, active_children
from tensorflow.keras import Input, layers, optimizers, backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from gsmmodem.modem import GsmModem
import os

In [3]:
#stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger('debugger')
logger.setLevel(logging.DEBUG)
ch = logging.FileHandler('spam.log')
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)
emit_string = "send message here"
logger.debug(emit_string)

## Variable Initializations

In [4]:
audio_format = pyaudio.paFloat32
audio_rate = 44100
audio_channels = 1
audio_device_index = 0
audio_frames_per_buffer = 4410
audio_sample_duration = 3
input_shape = (audio_rate, 1)
modem_port = '/dev/ttyUSB0'
modem_baudrate = 115200
modem_sim_pin = None # SIM card PIN (if any)
phone_numbers_to_message = ["8163449956", "9176202840", "7857642331"]

## Establishing a Connection to the SMS Modem

In [5]:
#print("Initializing connection to modem...")
#modem = GsmModem(modem_port, modem_baudrate)
#modem.smsTextMode = False
#modem.connect(modem_sim_pin)

## ROC (AUC) metric - Uses the import "from tensorflow.keras import backend as K"

In [6]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

## Original Model Parameters

In [7]:
drop_out_rate = 0.1
learning_rate = 0.001
number_of_epochs = 100
number_of_classes = 2
batch_size = 32
optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
input_tensor = Input(shape=input_shape)
metrics = [auc, "accuracy"]

Instructions for updating:
Colocations handled automatically by placer.


## Loading Original Model

In [8]:
x = layers.Conv1D(16, 9, activation="relu", padding="same")(input_tensor)
x = layers.Conv1D(16, 9, activation="relu", padding="same")(x)
x = layers.MaxPool1D(16)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.MaxPool1D(4)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.MaxPool1D(4)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dropout(rate=(drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(1028, activation="relu")(x)
output_tensor = layers.Dense(number_of_classes, activation="softmax")(x)

model = tf.keras.Model(input_tensor, output_tensor)
model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=metrics)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


## Loading Original Model Weights

In [9]:
print(os.getcwd())

/Users/gabe/Desktop/gunshot_detection/raspberry_pi


In [10]:
model.load_weights("/Users/gabe/Desktop/gunshot_detection/raspberry_pi/models/gunshot_sound_model_normalized.h5")

## Defining Thread Functions

In [11]:
def analyze_microphone_data(microphone_data, audio_rate, model, phone_numbers_to_message):
    # Performs post-processing on live audio samples
    reformed_microphone_data = librosa.resample(y=microphone_data, orig_sr=audio_rate, target_sr=22050)
    reformed_microphone_data = librosa.util.normalize(reformed_microphone_data)
    reformed_microphone_data = reformed_microphone_data[:audio_rate]
    reformed_microphone_data = reformed_microphone_data.reshape(-1, audio_rate, 1)
        
    # Passes a given audio sample into the model for prediction
    probabilities = model.predict(reformed_microphone_data)
    emit_string = "Probabilities derived by the model: " + str(probabilities)
    logger.debug(emit_string)
    if (probabilities[0][1] >= 0.9):
        sms_alert_process = Process(target = send_sms_alert, args = (probabilities))
        sms_alert_process.start()


def send_sms_alert(probabilities):
        # If the model detects a gunshot, an SMS alert will be sent to local authorities
        emit_string = "send message here"
        logger.debug(emit_string)
        #try:
        #    modem.waitForNetworkCoverage(timeout=86400)
        #    message = " (Testing) ALERT: A Gunshot Has Been Detected (Testing)"
        #    for number in phone_numbers_to_message:
        #        modem.sendSms(number, message)
        #    stream_handler.emit(" *** Sent out an SMS alert to all designated recipients *** ")
        #except:
        #    stream_handler.emit("ERROR: Unable to successfully send an SMS alert to the designated recipients.")
        #    pass
        #finally:
        #    stream_handler.emit(" * Finished evaluating an audio sample with the model * ")

## Capturing Microphone Audio

In [ ]:
pa = pyaudio.PyAudio()
    
stream = pa.open(format = audio_format,
                 rate = audio_rate,
                 channels = audio_channels,
                 input_device_index = audio_device_index,
                 frames_per_buffer = audio_frames_per_buffer,
                 input = True)

emit_string = "--- Recording Audio ---"
logger.debug(emit_string)
while(True):
    np_array_data = []
    
    # Loops through the stream and appends audio chunks to the frame array
    for i in range(0, int(audio_rate / audio_frames_per_buffer * audio_sample_duration)):
        data = stream.read(audio_frames_per_buffer, exception_on_overflow = False)
        np_array_data.append(np.frombuffer(data, dtype=np.float32))
    microphone_data = np.concatenate(np_array_data)
    #emit_string = "Cumulative length of a given two-second audio sample: " + str(len(microphone_data))
    logger.debug(emit_string)
    emit_string = "The maximum frequency value for a given two-second audio sample: " + str(max(microphone_data))
    #logger.debug(emit_string)
    
    # If a sample meets a certain threshold, a new concurrent analysis process is created
    if max(microphone_data) >= 0.001:
        logger.debug("made it")
        analysis_process = Process(target = analyze_microphone_data, args = (microphone_data, audio_rate, model, phone_numbers_to_message))
        analysis_process.start()
        
    # Closes all finished processes    
    active_children()